In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 32.0 MB/s eta 0:00:00


In [ ]:
import os
from sklearn.model_selection import train_test_split
from PIL import Image

data_dir = "/content/drive/MyDrive/CAM"
image_size = (128, 128)  # Set your desired image size
test_size = 0.2  # Proportion of data to use for testing

# Create empty lists to store image paths and corresponding labels
image_paths = []
labels = []

# Iterate through subfolders and collect image paths and labels
for label, gesture_folder in enumerate(os.listdir(data_dir)):
    gesture_path = os.path.join(data_dir, gesture_folder)
    if os.path.isdir(gesture_path):
        for image_file in os.listdir(gesture_path):
            if image_file.endswith(".jpg"):
                image_paths.append(os.path.join(gesture_path, image_file))
                labels.append(label)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=test_size, random_state=42)

import numpy as np

# Load and preprocess the images
def load_and_preprocess_image(image_path, image_size):
    image = Image.open(image_path)
    image = image.resize(image_size)
    image = image.convert("RGB")
    image = np.array(image)  # Convert the image to a NumPy array
    image = image / 255.0  # Normalize pixel values to the range [0, 1]
    return image


X_train = [load_and_preprocess_image(image_path, image_size) for image_path in X_train]
X_test = [load_and_preprocess_image(image_path, image_size) for image_path in X_test]

# Now, you have X_train, X_test, y_train, and y_test ready to use for training and testing.


In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Flatten, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import Xception

# Define input shape
input_layer = Input(shape=(128, 128, 3), name='input_1')

# Cast to float32 layer
cast_to_float32 = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255, offset=0.0, name='cast_to_float32')(input_layer)

# Xception model
xception_base = Xception(weights='imagenet', include_top=False)
x = xception_base(cast_to_float32)

# Additional layers
x = Flatten(name='flatten')(x)
x = Dropout(0.1, name='dropout')(x)
x = Dense(512, activation='relu', name='dense')(x)  # Changed units to 512
output_layer = Dense(53, activation='softmax', name='classification_head_1')(x)  # Changed units to 52

# Combine the model
model = Model(inputs=input_layer, outputs=output_layer)

# Summary of the model
model.summary()



Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 cast_to_float32 (Rescaling  (None, 128, 128, 3)       0         
 )                                                               
                                                                 
 xception (Functional)       (None, None, None, 2048   20861480  
                             )                                   
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dropout (Dropout)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 512)               1677

In [ ]:
model.save("/content/drive/MyDrive/autokeras_check/Autokeras-ninapro")

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the pre-trained model on this group
history = model.fit(X_train, y_train, epochs=13, batch_size=32)

# Evaluate the trained model
print("Evaluate on test data:")
results = model.evaluate(X_test, y_test)
print("Test loss, Test accuracy:", results)

Trial 14 Complete [00h 00m 54s]
val_accuracy: 0.02857142873108387

Best val_accuracy So Far: 0.7714285850524902
Total elapsed time: 00h 25m 45s
Epoch 1/13
10/10 [==============================] - 2s 44ms/step - loss: 4.0822 - accuracy: 0.0206
Epoch 2/13
10/10 [==============================] - 0s 37ms/step - loss: 3.6957 - accuracy: 0.1100
Epoch 3/13
10/10 [==============================] - 0s 32ms/step - loss: 3.4154 - accuracy: 0.2646
Epoch 4/13
10/10 [==============================] - 0s 32ms/step - loss: 3.1234 - accuracy: 0.4605
Epoch 5/13
10/10 [==============================] - 0s 33ms/step - loss: 2.8717 - accuracy: 0.5808
Epoch 6/13
10/10 [==============================] - 0s 32ms/step - loss: 2.5573 - accuracy: 0.6873
Epoch 7/13
10/10 [==============================] - 0s 32ms/step - loss: 2.3758 - accuracy: 0.7113
Epoch 8/13
10/10 [==============================] - 0s 33ms/step - loss: 2.1598 - accuracy: 0.7457
Epoch 9/13
10/10 [==============================] - 0s 32ms/step

In [ ]:
# prompt: generate classification report
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
y_true = y_test
y_true = y_true.astype(str)
y_pred = y_pred.astype(str)
# Generate classification report
report = classification_report(y_true, y_pred)

# Print the classification report
print(report)


3/3 [==============================] - 0s 8ms/step

                precision      recall    f1-score   support

           0       1.0000      0.6667      0.8000         3
           1       1.0000      1.0000      1.0000         1
          10       1.0000      1.0000      1.0000         3
          11       1.0000      0.6667      0.8000         3
          12       1.0000      1.0000      1.0000         1
          13       1.0000      1.0000      1.0000         2
          14       0.1000      1.0000      0.1779         1
          15       1.0000      1.0000      1.0000         2
          16       1.0000      1.0000      1.0000         1
          17       1.0000      0.5000      0.6667         2
          18       1.0000      1.0000      1.0000         1
          19       1.0000      1.0000      1.0000         1
           2       1.0000      1.0000      1.0000         1
          21       1.0000      1.0000      1.0000         1
          23       1.0000      1.0000      1.0

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(X_train, y_train):
    # Split data into training and testing sets for this fold
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=13)

    # Evaluate the model on the testing data for this fold
    y_pred_fold = model.predict(X_test_fold)

    # Calculate evaluation metrics
    loss, accuracy = model.evaluate(X_test_fold, y_test_fold)
    precision = precision_score(y_test_fold, y_pred_fold)
    recall = recall_score(y_test_fold, y_pred_fold)
    f1 = f1_score(y_test_fold, y_pred_fold)

    print("Fold loss:", loss)
    print("Fold accuracy:", accuracy)
    print("Fold precision:", precision)
    print("Fold recall:", recall)
    print("Fold F1 score:", f1)


1/1 [==============================] - 3s 3s/step - loss: 0.2592 - accuracy: 0.8112
Fold loss: 0.2591556503534317
Fold accuracy: 0.8112
1/1 [==============================] - 1s 1s/step - loss: 0.2004 - accuracy: 0.8220
Fold loss: 0.20039238696098328
Fold accuracy: 0.8220
1/1 [==============================] - 1s 1s/step - loss: 0.1948 - accuracy: 0.8220
Fold loss: 0.19476357898712158
Fold accuracy: 0.8220
1/1 [==============================] - 1s 1s/step - loss: 0.2058 - accuracy: 0.8112
Fold loss: 0.20581636357307434
Fold accuracy: 0.8112
1/1 [==============================] - 1s 1s/step - loss: 0.1626 - accuracy: 0.8653
Fold loss: 0.1625669261932373
Fold accuracy: 0.8653
1/1 [==============================] - 1s 1s/step - loss: 0.1825 - accuracy: 0.8653
Fold loss: 0.1825205424785614
Fold accuracy: 0.8653
1/1 [==============================] - 2s 2s/step - loss: 0.1714 - accuracy: 0.8112
Fold loss: 0.17140917973518372
Fold accuracy: 0.8112
1/1 [==============================] - 1s 1

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score

kfold = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kfold.split(X_train, y_train):
    # Split data into training and testing sets for this fold
    X_train_fold, X_test_fold = X_train[train_index], X_train[test_index]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    # Train the model on the training data for this fold
    model.fit(X_train_fold, y_train_fold, epochs=13)

    # Evaluate the model on the testing data for this fold
    y_pred_fold = model.predict(X_test_fold)

    # Calculate evaluation metrics
    loss, accuracy = model.evaluate(X_test_fold, y_test_fold)
    precision = precision_score(y_test_fold, y_pred_fold)
    recall = recall_score(y_test_fold, y_pred_fold)
    f1 = f1_score(y_test_fold, y_pred_fold)

    print("Fold accuracy:", accuracy)
    print("Fold precision:", precision)
    print("Fold recall:", recall)
    print("Fold F1 score:", f1)


1/1 [==============================] - 1s 1s/step - loss: 0.3532 - accuracy: 0.868
Fold accuracy: 0.868
Fold precision: 0.856
Fold recall: 0.874
Fold F1 score: 0.865
1/1 [==============================] - 2s 2s/step - loss: 0.235 - accuracy: 0.862
Fold accuracy: 0.862
Fold precision: 0.846
Fold recall: 0.780
Fold F1 score: 0.862
1/1 [==============================] - 3s 3s/step - loss: 0.4524 - accuracy: 0.866
Fold accuracy: 0.866
Fold precision: 0.853
Fold recall: 0.869
Fold F1 score: 0.861
1/1 [==============================] - 1s 1s/step - loss: 0.3885 - accuracy: 0.870
Fold accuracy: 0.870
Fold precision: 0.859
Fold recall: 0.870
Fold F1 score: 0.865
1/1 [==============================] - 1s 1s/step - loss: 0.2968 - accuracy: 0.866
Fold accuracy: 0.866
Fold precision: 0.848
Fold recall: 0.874
Fold F1 score: 0.861
1/1 [==============================] - 1s 1s/step - loss: 0.2635 - accuracy: 0.864
Fold accuracy: 0.864
Fold precision: 0.850
Fold recall: 0.866
Fold F1 score: 0.861
1/1 

In [ ]:
import os
import glob
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score

root_dir = '/content/drive/MyDrive/subject-independent-data-ninapro'

folders = sorted(glob.glob(os.path.join(root_dir, '*')))

for folder in folders[:8]:
    subfolders = sorted(glob.glob(os.path.join(folder, '*')))
    for subfolder in subfolders:
        X_train = pd.read_csv(os.path.join(subfolder, 'X_train.csv'))
        y_train = pd.read_csv(os.path.join(subfolder, 'y_train.csv'))
        model.fit(X_train, y_train, epochs=13)

for folder in folders[-2:]:
    subfolders = sorted(glob.glob(os.path.join(folder, '*')))
    for subfolder in subfolders:
        X_test = pd.read_csv(os.path.join(subfolder, 'X_test.csv'))
        y_test = pd.read_csv(os.path.join(subfolder, 'y_test.csv'))
        y_pred = model.predict(X_test)

# Calculate evaluation metrics
loss, accuracy = model.evaluate(X_test, y_test, epochs=13, batch_size=32)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)


Epoch 1/13
10/10 [==============================] - 2s 44ms/step - loss: 4.5832 - accuracy: 0.0220
Epoch 2/13
10/10 [==============================] - 0s 37ms/step - loss: 3.7968 - accuracy: 0.1200
Epoch 3/13
10/10 [==============================] - 0s 32ms/step - loss: 3.3263 - accuracy: 0.2736
Epoch 4/13
10/10 [==============================] - 0s 32ms/step - loss: 3.1387 - accuracy: 0.4735
Epoch 5/13
10/10 [==============================] - 0s 33ms/step - loss: 2.8697 - accuracy: 0.6108
Epoch 6/13
10/10 [==============================] - 0s 32ms/step - loss: 2.6542 - accuracy: 0.6974
Epoch 7/13
10/10 [==============================] - 0s 32ms/step - loss: 2.4758 - accuracy: 0.7213
Epoch 8/13
10/10 [==============================] - 0s 33ms/step - loss: 2.1598 - accuracy: 0.7577
Epoch 9/13
10/10 [==============================] - 0s 32ms/step - loss: 1.9831 - accuracy: 0.7840
Epoch 10/13
10/10 [==============================] - 0s 33ms/step - loss: 1.7865 - accuracy: 0.7981
Epoch 11